## Requirements

In [1]:
import random

import open3d as o3d
from joblib import load, dump

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from scipy.special import softmax
import numpy as np

import torch
import scipy.spatial.distance
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch import nn
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
random.seed = 42

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
rf_point = load("./model_weights/rf_point.joblib")

# PointNet model

In [4]:
# Used for extracting feature rich vector giving a 1 dimensional vector for point net

class Tnet(nn.Module):
   def __init__(self, k=3):
      super().__init__()
      self.k=k
      self.conv1 = nn.Conv1d(k,64,1)
      self.conv2 = nn.Conv1d(64,128,1)
      self.conv3 = nn.Conv1d(128,1024,1)
      self.fc1 = nn.Linear(1024,512)
      self.fc2 = nn.Linear(512,256)
      self.fc3 = nn.Linear(256,k*k)

      self.bn1 = nn.BatchNorm1d(64)
      self.bn2 = nn.BatchNorm1d(128)
      self.bn3 = nn.BatchNorm1d(1024)
      self.bn4 = nn.BatchNorm1d(512)
      self.bn5 = nn.BatchNorm1d(256)
       

   def forward(self, input):
      # input.shape == (bs,n,3)
      bs = input.size(0)
      xb = F.relu(self.bn1(self.conv1(input)))
      xb = F.relu(self.bn2(self.conv2(xb)))
      xb = F.relu(self.bn3(self.conv3(xb)))
      pool = nn.MaxPool1d(xb.size(-1))(xb)
      flat = nn.Flatten(1)(pool)
      xb = F.relu(self.bn4(self.fc1(flat)))
      xb = F.relu(self.bn5(self.fc2(xb)))
      
      #initialize as identity
      init = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
      if xb.is_cuda:
        init=init.cuda()
      matrix = self.fc3(xb).view(-1,self.k,self.k) + init
      return matrix

# Used for position estimation and point estimation using global and
# local coordinates
class Transform(nn.Module):
   def __init__(self):
        super().__init__()
        self.input_transform = Tnet(k=3)
        self.feature_transform = Tnet(k=64)
        self.conv1 = nn.Conv1d(3,64,1)#(3,64,1)

        self.conv2 = nn.Conv1d(64,128,1)
        self.conv3 = nn.Conv1d(128,1024,1)
       

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
       
   def forward(self, input):
        matrix3x3 = self.input_transform(input)
        # batch matrix multiplication
        xb = torch.bmm(torch.transpose(input,1,2), matrix3x3).transpose(1,2)

        xb = F.relu(self.bn1(self.conv1(xb)))

        matrix64x64 = self.feature_transform(xb)
        xb = torch.bmm(torch.transpose(xb,1,2), matrix64x64).transpose(1,2)

        xb = F.relu(self.bn2(self.conv2(xb)))
        xb = self.bn3(self.conv3(xb))
        xb = nn.MaxPool1d(xb.size(-1))(xb)
        output = nn.Flatten(1)(xb)
        return output, matrix3x3, matrix64x64

class PointNet(nn.Module):
    def __init__(self, classes = 2):
        super().__init__()
        self.transform = Transform()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, classes)
        

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.3)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, input):
        xb, matrix3x3, matrix64x64 = self.transform(input)
        xb1 = F.relu(self.bn1(self.fc1(xb)))
        xb2 = F.relu(self.bn2(self.dropout(self.fc2(xb1))))
        output = self.fc3(xb2)
        return self.logsoftmax(output), matrix3x3, matrix64x64, xb1, xb2 

# PointNet custom loss calculating with all 3 model outputs

# LOSS
def pointnetloss(outputs, labels, m3x3, m64x64, alpha = 0.0001):
    criterion = torch.nn.NLLLoss()
    bs = outputs.size(0)
    id3x3 = torch.eye(3, requires_grad=True).repeat(bs, 1, 1)
    id64x64 = torch.eye(64, requires_grad=True).repeat(bs, 1, 1)
    if outputs.is_cuda:
        id3x3 = id3x3.cuda()
        id64x64 = id64x64.cuda()
    diff3x3 = id3x3 - torch.bmm(m3x3, m3x3.transpose(1, 2))
    diff64x64 = id64x64 - torch.bmm(m64x64, m64x64.transpose(1, 2))
    return criterion(outputs, labels) + alpha * (torch.norm(diff3x3) + torch.norm(diff64x64)) / float(bs)

# Normalization used as a preprocessing step
class Normalize(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        
        norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0) 
        norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

        return  norm_pointcloud

def default_transforms():
    return transforms.Compose([
                                Normalize(),
                                
                              ])

In [8]:
class PointCloudDataRFSampler(Dataset):
    def __init__(self, dataframe_path, sample_rate=1024, transform=default_transforms()):
        self.df = pd.read_csv(dataframe_path)
        self.classes = {"anomaly": 1, "normal": 0}
        self.sample_rate=sample_rate
        self.transforms=transform

    def __len__(self):
        return self.df.shape[0]

    def __preproc__(self, file):#, reference_path):
        # Loads point cloud
        resampled = pd.read_csv(file).values
        # Apply transforms (only normalize but additional can be added)
        if self.transforms:
            pointcloud = self.transforms(resampled)
        return torch.from_numpy(pointcloud)

    def __getitem__(self, idx):
        pcd_path = self.df.iloc[idx]['sampled_pcds']
        # ref_path = self.df.iloc[idx]['reference_path']
        category = self.df.iloc[idx]['label']
        pointcloud = self.__preproc__(pcd_path)#, ref_path)
        return {'pointcloud': pointcloud, 
                'category': self.classes[category]}



In [9]:
train_point_ds = PointCloudDataRFSampler("./data/RF_sampled_df.csv", sample_rate=1024)
test_point_ds = PointCloudDataRFSampler("./data/RF_sampled_df_test.csv", sample_rate=1024)

print(len(train_point_ds))
print(len(test_point_ds))


train_loader = DataLoader(dataset=train_point_ds, batch_size=32)
test_loader = DataLoader(dataset=test_point_ds, batch_size=32)

32
8


In [10]:
batch_size = 32
learning_rate = 0.0001
sample_rate = 1024
epochs = 10

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

pointnet = PointNet()
pointnet.to(device)
optimizer = torch.optim.Adam(pointnet.parameters(), lr=learning_rate)

pointnet.load_state_dict(torch.load("./model_weights/PointNet_RF_samples_0.pth")) # Load best fitting model
pointnet.eval();

cpu


In [11]:
%%capture

# Calculate embeddings for use in random forest mesh classifier
rf_train_level_1_embs = []
rf_train_level_2_embs = []
with torch.no_grad():
    for i, data in enumerate(train_loader):
        print('Batch [%4d / %4d]' % (i+1, len(train_loader)))
                   
        pcd = data['pointcloud'].to(device).float()
        labels = data['category'].to(device)
        optimizer.zero_grad()

        _, _, _, emb1, emb2 = pointnet(pcd.transpose(1,2))
        rf_train_level_1_embs += list(emb1.cpu().numpy())
        rf_train_level_2_embs += list(emb2.cpu().numpy())


rf_test_level_1_embs = []
rf_test_level_2_embs = []
with torch.no_grad():
    for i, data in enumerate(test_loader):
        print('Batch [%4d / %4d]' % (i+1, len(test_loader)))
        
        pcd = data['pointcloud'].to(device).float()
        labels = data['category'].to(device)
        optimizer.zero_grad()

        _, _, _, emb1, emb2 = pointnet(pcd.transpose(1,2))
        # _, preds = torch.max(outputs.data, 1)
        rf_test_level_1_embs += list(emb1.cpu().numpy())
        rf_test_level_2_embs += list(emb2.cpu().numpy())

In [13]:
# Load aggregated percentile data
train_anomaly_mesh = pd.read_csv("./data/aggregated_train.csv")
test_anomaly_mesh = pd.read_csv("./data/aggregated_test.csv")

# Take embeddings from PointNet of shape 512 (lvl1) and 256 (lvl2)
# Concatenate embedding and percentile vectors
rf_full_test_dataset = []
for aggr_file, np_emb in zip(test_anomaly_mesh.iterrows(), rf_test_level_1_embs):
  rf_full_test_dataset.append(np.concatenate([aggr_file[1].drop(["label_obj_is_anom"]).values, np_emb]))

rf_full_test_dataset_lvl2 = []
for aggr_file, np_emb in zip(test_anomaly_mesh.iterrows(), rf_test_level_2_embs):
  rf_full_test_dataset_lvl2.append(np.concatenate([aggr_file[1].drop(["label_obj_is_anom"]).values, np_emb]))

rf_full_train_dataset = []
for aggr_file, np_emb in zip(train_anomaly_mesh.iterrows(), rf_train_level_1_embs):
  rf_full_train_dataset.append(np.concatenate([aggr_file[1].drop(["label_obj_is_anom"]).values, np_emb]))

rf_full_train_dataset_lvl2 = []
for aggr_file, np_emb in zip(train_anomaly_mesh.iterrows(), rf_train_level_2_embs):
  rf_full_train_dataset_lvl2.append(np.concatenate([aggr_file[1].drop(["label_obj_is_anom"]).values, np_emb]))



In [15]:
train_df = pd.read_csv("./data/train_df.csv")
test_df = pd.read_csv("./data/test_df.csv")

In [16]:
#Train Random Forest model on embeddings 
rf_sample_features = np.array(rf_full_train_dataset)
rf_labels = np.array([1 if val == "anomaly" else 0 for val in train_df['label']])
rf_mesh_rf_sample = RandomForestClassifier()
rf_mesh_rf_sample.fit(rf_sample_features, rf_labels)

test_rf_sample_features = np.array(rf_full_test_dataset)
test_rf_labels = np.array([1 if val == "anomaly" else 0 for val in test_df['label']])

test_preds = rf_mesh_rf_sample.predict(test_rf_sample_features)
cm = confusion_matrix(test_rf_labels, test_preds);
print(cm)

[[3 1]
 [3 1]]


In [17]:
# Level2 embedding random forest
rf_sample_features_lvl2 = np.array(rf_full_train_dataset_lvl2)
rf_labels = np.array([1 if val == "anomaly" else 0 for val in train_df['label']])

rf_mesh_rf_sample_lvl2 = RandomForestClassifier()
rf_mesh_rf_sample_lvl2.fit(rf_sample_features_lvl2, rf_labels)

test_rf_labels_lvl2 = np.array([1 if val == "anomaly" else 0 for val in test_df['label']])

test_rf_sample_features_lvl2 = np.array(rf_full_test_dataset_lvl2)

test_preds_lvl2 = rf_mesh_rf_sample_lvl2.predict(test_rf_sample_features_lvl2)
cm = confusion_matrix(test_rf_labels_lvl2, test_preds_lvl2);
print(cm)

[[1 3]
 [1 3]]


In [19]:
def get_inds(df_90_10, full_df):
  inds = []
  for idx, filename in enumerate(full_df["object_path"]):
    if filename in df_90_10["object_path"].tolist():
      inds.append(idx)
  return inds

# Get indices used of 90/10 indices and extract the points for the relevant indices
test_anomaly_mesh_90_10 = test_anomaly_mesh.iloc[get_inds(pd.read_csv("./data/test_df_90_10.csv"), test_df)]

test_dataset_90_10_lvl1 = []
test_dataset_90_10_lvl2 = []
for i in range(test_anomaly_mesh_90_10.shape[0]):
  idx = test_anomaly_mesh_90_10.index[i]
  test_dataset_90_10_lvl1.append(np.concatenate([test_anomaly_mesh_90_10.iloc[i].drop(["label_obj_is_anom"]).values, 
                                                 rf_test_level_1_embs[idx]]))
  test_dataset_90_10_lvl2.append(np.concatenate([test_anomaly_mesh_90_10.iloc[i].drop(["label_obj_is_anom"]).values, 
                                                 rf_test_level_2_embs[idx]]))

test_labels_90_10 = test_anomaly_mesh_90_10["label_obj_is_anom"].astype(int)
test_dataset_90_10_lvl1 = np.array(test_dataset_90_10_lvl1)
test_dataset_90_10_lvl2 = np.array(test_dataset_90_10_lvl2)

preds_90_10 = rf_mesh_rf_sample.predict(test_dataset_90_10_lvl1)
cm_90_10 = confusion_matrix(test_labels_90_10, preds_90_10);
print(cm_90_10)  

preds_lvl2_90_10 = rf_mesh_rf_sample_lvl2.predict(test_dataset_90_10_lvl2)
cm_lvl2_90_10 = confusion_matrix(test_labels_90_10, preds_lvl2_90_10);
print(cm_lvl2_90_10)


[[3 1]
 [0 0]]
[[1 3]
 [0 0]]


In [20]:
# Save model and transfer to drive

dump(rf_mesh_rf_sample, "./model_weights/rf_mesh+PointNet+RFSampling_emb_512.joblib")
dump(rf_mesh_rf_sample_lvl2, "./model_weights/rf_mesh+PointNet+RFSampling_emb_256m.joblib")


['./model_weights/rf_mesh+PointNet+RFSampling_emb_256m_50-50.joblib']